In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from sklearn.model_selection import KFold
from torchvision import transforms
from chart_utils import TimeSeriesImageDataset, accuracy_fn

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Function to read UCR dataset
def read_ucr(filename):
    data = []
    labels = []
    
    with open(filename, 'r') as file:
        for line in file:
            parts = line.strip().split(',')
            if len(parts) < 2:
                continue
            features = [float(f) for f in parts[:-1]]
            label = int(parts[-1].split(':')[-1])
            data.append(features)
            labels.append(label)
    
    print(f"Loaded {len(data)} samples from {filename}")
    return np.array(data), np.array(labels)

# File paths 
train_file = 'ECG5000/ECG5000_TRAIN.ts'
test_file = 'ECG5000/ECG5000_TEST.ts'

# Load dataset
x_train, y_train = read_ucr(train_file)
x_test, y_test = read_ucr(test_file)

# Normalize labels to be within range [0, num_classes-1]
unique_labels = np.unique(y_train)
label_map = {label: idx for idx, label in enumerate(unique_labels)}
y_train = np.array([label_map[label] for label in y_train])
y_test = np.array([label_map[label] for label in y_test])

nb_classes = len(unique_labels)

# Normalize features
x_train_mean = x_train.mean()
x_train_std = x_train.std()
x_train = (x_train - x_train_mean) / x_train_std
x_test = (x_test - x_train_mean) / x_train_std

# Applying SMOTE for resampling training data
smote = SMOTE()
x_train_resampled, y_train_resampled = smote.fit_resample(x_train, y_train)

# Convert to PyTorch tensors
X_train = torch.tensor(x_train_resampled, dtype=torch.float32)
X_test = torch.tensor(x_test, dtype=torch.float32)
y_train = torch.tensor(y_train_resampled, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

# Define a simpler CNN Model
class Simple1DCNN(nn.Module):
    def __init__(self, input_channels, num_classes):
        super(Simple1DCNN, self).__init__()
        self.conv1 = nn.Conv2d(input_channels, 16, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.dropout1 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(16 * 32 * 32, 64)
        self.dropout2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, num_classes)

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.bn1(self.conv1(x))))
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

# Data Augmentation
transform = transforms.Compose([
    transforms.Resize((64, 64)),  # Resize to a smaller size for simpler model
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2)
])

# Creating dataloaders    
train_dataset = TimeSeriesImageDataset(train_data.numpy(), train_labels.numpy(), transform)
test_dataset = TimeSeriesImageDataset(test_data.numpy(), test_labels.numpy(), transform)
    
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    
# Initialize the model, criterion, optimizer, and scheduler
model = Simple1DCNN(6, nb_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.01)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)
    
num_epochs = 10
best_test_accuracy = 0
patience = 10
trigger_times = 0

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images_area, images_bar, labels in train_loader:
        images_area, images_bar, labels = images_area.to(device), images_bar.to(device), labels.to(device)
            
         # Combine area and bar charts along the channel dimension
        combined_images = torch.cat((images_area, images_bar), dim=1)
            
        optimizer.zero_grad()
        outputs = model(combined_images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_accuracy:.2f}%')

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images_area, images_bar, labels in test_loader:
            images_area, images_bar, labels = images_area.to(device), images_bar.to(device), labels.to(device)
                
            # Combine area and bar charts along the channel dimension
            combined_images = torch.cat((images_area, images_bar), dim=1)
                
            outputs = model(combined_images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    test_accuracy = 100 * correct / total
    print(f'Test Accuracy after Epoch {epoch+1}: {test_accuracy:.2f}%')
    scheduler.step(epoch_loss)

    if test_accuracy > best_test_accuracy:
        best_test_accuracy = test_accuracy
        trigger_times = 0
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print(f'Early stopping at epoch {epoch+1}')
            break

print(f'Best Test Accuracy: {best_test_accuracy:.2f}%')
   


Loaded 500 samples from ECG5000/ECG5000_TRAIN.ts
Loaded 4500 samples from ECG5000/ECG5000_TEST.ts
Epoch [1/10], Loss: 1.2977, Training Accuracy: 68.38%
Test Accuracy after Epoch 1: 87.30%


C:\Users\vmadhuu\AppData\Local\anaconda3\envs\tscproj\Lib\site-packages\torch\optim\lr_scheduler.py:156: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch [2/10], Loss: 0.8679, Training Accuracy: 72.97%
Test Accuracy after Epoch 2: 91.20%
Epoch [3/10], Loss: 0.7546, Training Accuracy: 74.65%
Test Accuracy after Epoch 3: 91.30%
Epoch [4/10], Loss: 0.7319, Training Accuracy: 75.17%
Test Accuracy after Epoch 4: 92.20%
Epoch [5/10], Loss: 0.6644, Training Accuracy: 77.15%
Test Accuracy after Epoch 5: 90.40%
Epoch [6/10], Loss: 0.6555, Training Accuracy: 78.45%
Test Accuracy after Epoch 6: 91.90%
Epoch [7/10], Loss: 0.6635, Training Accuracy: 77.85%
Test Accuracy after Epoch 7: 93.00%
Epoch [8/10], Loss: 0.6499, Training Accuracy: 77.88%
Test Accuracy after Epoch 8: 90.60%
Epoch [9/10], Loss: 0.6480, Training Accuracy: 78.92%
Test Accuracy after Epoch 9: 91.50%
Epoch [10/10], Loss: 0.6470, Training Accuracy: 78.70%
Test Accuracy after Epoch 10: 91.70%
Best Test Accuracy: 93.00%
